In [7]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB
import gurobipy as gp
import coptpy as copt

# Slect solver : "copt" or "gurobi"
solver = "copt"

In [8]:
# 读取Result方案的数据，计算指标
# 读取数据
file_path = 'Result.xlsx'
result_data = pd.ExcelFile(file_path)
result_benchmark = result_data.parse('甘特图', index_col=0)

In [9]:
# Load data
file_path = 'Data.xlsx'
data = pd.ExcelFile(file_path)

# Read all necessary sheets
distance_data = data.parse('待排交路信息', index_col=0)
train_data = data.parse('车组里程修时信息', index_col=0)
repair_capability = data.parse('班组检修能力', index_col=0)
candidate_data = data.parse('候选交路', header=None, index_col=0)
restore_info = data.parse('车组修后恢复信息', index_col=0)
day1_schedule = data.parse('Day1检修上线情况', index_col=0)

In [10]:
candidate_data.columns = ['1', '2', '3', '4', '5', '6', '7', '8']
# 转换为字典集合，key 为对应行的索引
candidate_routes = {index: set(filter(pd.notna, row[:])) for index, row in candidate_data.iterrows()}
print(candidate_routes)

{'V001': {'R12', 'R11', 'R6', 'R9', 'R5', 'R10', 'R8', 'R7'}, 'V002': {'R1', 'R3', 'R2', 'R5', 'R4'}, 'V003': {'R12', 'R11', 'R13', 'R9', 'R5', 'R10', 'R8', 'R7'}, 'V004': {'R1', 'R6', 'R3', 'R2', 'R5', 'R4'}, 'V005': {'R1', 'R3', 'R2', 'R5', 'R4'}, 'V006': {'R1', 'R6', 'R3', 'R2', 'R5', 'R4'}, 'V007': {'R4', 'R6', 'R9', 'R5', 'R10', 'R8', 'R7'}, 'V008': {'R4', 'R6', 'R3', 'R2', 'R5', 'R8', 'R7'}, 'V009': {'R11', 'R6', 'R9', 'R5', 'R10', 'R8', 'R7'}, 'V010': {'R12', 'R11', 'R13', 'R9', 'R5', 'R10', 'R8', 'R7'}, 'V011': {'R1', 'R6', 'R3', 'R2', 'R5', 'R4'}, 'V012': {'R1', 'R4', 'R6', 'R3', 'R2', 'R5', 'R7'}, 'V013': {'R4', 'R6', 'R3', 'R2', 'R5', 'R8', 'R7'}, 'V014': {'R12', 'R14', 'R11', 'R13', 'R9', 'R5', 'R10', 'R8'}, 'V015': {'R4', 'R6', 'R3', 'R2', 'R5', 'R8', 'R7'}, 'V016': {'R17', 'R1', 'R6', 'R9', 'R5', 'R18', 'R8', 'R7'}, 'V017': {'R12', 'R14', 'R11', 'R13', 'R5', 'R15', 'R10', 'R16'}, 'V018': {'R17', 'R14', 'R13', 'R5', 'R15', 'R18', 'R16'}, 'V019': {'R12', 'R14', 'R11', 'R13'

In [11]:
reset_distance = {}
reset_days = {}
reset_distance['Z'] = restore_info.loc['Z']['修后恢复公里数']
reset_days['Z'] = restore_info.loc['Z']['修后恢复天数']
reset_distance['L'] = restore_info.loc['L']['修后恢复公里数']
print(reset_distance)
print(reset_days)

{'Z': np.float64(66000.0), 'L': np.float64(250000.0)}
{'Z': np.float64(66.0)}


In [12]:
# Parameters
# 交路转换为字典集合，key 为 R_ID，value 为对应行的索引列表
route_crosses = {key: list(value) for key, value in distance_data.groupby('R_ID').groups.items()}
print(route_crosses)
cross_routes = list(distance_data.index)
print(cross_routes)
routes_set =  list(set(distance_data['R_ID'].unique()))
print(routes_set)
train_set =  list(set(train_data.index.unique()))
print(train_set)
# 将列名中的 'day' 替换为空字符串，并转换为列表
days_list = [col.replace('day', '') for col in repair_capability.columns]
print(days_list)
num_trains = len(train_set)
print(num_trains)
num_days = len(days_list) 
print(num_days)
num_routes = len(routes_set)
print(num_routes)
repair_types = ['Z', 'L']

{'R1': ['r1'], 'R10': ['r17', 'r18'], 'R11': ['r19', 'r20', 'r21'], 'R13': ['r22', 'r23', 'r24'], 'R14': ['r25', 'r26'], 'R15': ['r27', 'r28'], 'R16': ['r29', 'r30'], 'R17': ['r31', 'r32'], 'R18': ['r33', 'r34', 'r35'], 'R2': ['r2', 'r3'], 'R3': ['r4', 'r5', 'r6'], 'R4': ['r7'], 'R5': ['r8', 'r9', 'r10'], 'R6': ['r11', 'r12', 'r13'], 'R7': ['r14'], 'R8': ['r15'], 'R9': ['r16']}
['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35']
['R1', 'R14', 'R11', 'R17', 'R6', 'R18', 'R16', 'R3', 'R13', 'R9', 'R2', 'R5', 'R7', 'R15', 'R10', 'R8', 'R4']
['V001', 'V032', 'V014', 'V020', 'V033', 'V012', 'V009', 'V024', 'V018', 'V002', 'V010', 'V008', 'V007', 'V027', 'V022', 'V013', 'V029', 'V004', 'V017', 'V028', 'V034', 'V023', 'V019', 'V016', 'V005', 'V021', 'V026', 'V030', 'V006', 'V011', 'V015', 'V025', 'V003', 'V035'

In [13]:
def evaluate_schedule(result_df):
    # 1) 计算过度维修里程和天数
    def calculate_excess_maintenance(result_df):
        # 假设每个车组的初始里程和天数
        remain_distance_days = train_data.copy()
        
        # 初始化剩余里程和天数
        remaining_mileage = {type: 0 for type in repair_types}
        remaining_days = {type: 0 for type in repair_types}

        # 计算剩余里程和天数
        for day in range(1, num_days):
            # 车组任务
            for cross in cross_routes:
                train = result_df.loc[cross]['Day' + str(day + 1)]
                if train == '无任务':
                    continue
                remain_distance_days.loc[train, 'Z' + '剩余天数'] -= 1
                for type in repair_types:
                    remain_distance_days.loc[train, type + '剩余里程'] -= distance_data.loc[cross]['distance']
            # 检修任务
            for type in repair_types:
                if pd.isna(result_df.loc[type]['Day' + str(day + 1)]) or result_df.loc[type]['Day' + str(day + 1)] in ['', None]:
                    continue
                trains = result_df.loc[type]['Day' + str(day + 1)].split(',')
                for train in trains:
                    if type == 'Z':
                        remaining_days['Z'] += remain_distance_days.loc[train]['Z剩余天数']
                        remain_distance_days.loc[train, 'Z剩余天数'] = reset_days['Z']
                    remaining_mileage[type] += remain_distance_days.loc[train][type + '剩余里程']
                    remain_distance_days.loc[train, type + '剩余里程'] = reset_distance[type]
        
        return remaining_mileage, remaining_days

    # 2) 计算换车次数
    def calculate_switch_count(df):
        train_assigned_routes = {}
        for day in range(num_days):
            for cross in cross_routes:
                train = df.loc[cross]['Day' + str(day + 1)]
                if train == '无任务':
                    continue
                if train not in train_assigned_routes:
                    train_assigned_routes[train] = []
                    train_assigned_routes[train].append(distance_data.loc[cross]['R_ID'])
                elif distance_data.loc[cross]['R_ID'] not in train_assigned_routes[train]:
                    train_assigned_routes[train].append(distance_data.loc[cross]['R_ID'])

        switch_count = 0
        for train in train_assigned_routes.keys():
            switch_count += len(train_assigned_routes[train]) - 1
        return switch_count

    # 计算每天维修工作量的极差与方差
    def calculate_workload_variance(result_df):
        daily_workload = []
        for day in range(num_days):
            workload = 0
            for type in repair_types:
                if pd.isna(result_df.loc[type]['Day' + str(day + 1)]) or result_df.loc[type]['Day' + str(day + 1)] in ['', None]:
                    continue
                trains = result_df.loc[type]['Day' + str(day + 1)].split(',')
                workload += len(trains)
            daily_workload.append(workload)
        workload_range = max(daily_workload) - min(daily_workload)
        workload_variance = sum([(x - np.mean(daily_workload)) ** 2 for x in daily_workload]) / len(daily_workload)
        return workload_range, workload_variance

    # 计算评价指标
    remaining_mileage, remaining_days = calculate_excess_maintenance(result_df)
    switch_count = calculate_switch_count(result_df)
    workload_range, workload_variance = calculate_workload_variance(result_df)
        
    return {
            '过度维修Z里程': remaining_mileage['Z'],
            '过度维修L里程': remaining_mileage['L'],
            '过度维修Z天数': remaining_days['Z'],
            '累积换车次数': switch_count,
            '维修工作量极差': workload_range,
            '维修工作量方差': workload_variance
        }

In [14]:
if solver == 'copt':
    BINARY = copt.COPT.BINARY
    CONTINUOUS = copt.COPT.CONTINUOUS
    MINIMIZE = copt.COPT.MINIMIZE
    OPTIMAL = copt.COPT.OPTIMAL
    INFINITY = copt.COPT.INFINITY
    # Initialize model
    # Initialize model
    env = copt.Envr()
    model = env.createModel("TrainSchedule")
    # # 关闭日志输出
    model.setParam(copt.COPT.Param.Logging, 0)

elif solver == "gurobi":
    BINARY = gp.GRB.BINARY
    CONTINUOUS = gp.GRB.CONTINUOUS
    MINIMIZE = gp.GRB.MINIMIZE
    OPTIMAL = gp.GRB.OPTIMAL
    INFINITY = gp.GRB.INFINITY
    model = gp.Model('TrainSchedule')
    # # 关闭日志输出
    model.setParam('OutputFlag', 0)

# Variables
# x: train assignment to cross route on day : x[train, cross, day] = 1 if train assigned to cross route on day, 0 otherwise
x = {}
for day in range(num_days):
    for train in train_set:
        for route in candidate_routes[train]:
            if route not in route_crosses.keys():
                continue
            for cross in route_crosses[route]:
                if day == 0:
                    value = day1_schedule.loc[cross][train]
                    x[train, cross, day] = model.addVar(vtype = BINARY, lb = value, ub = value, name=f'x_{train}_{cross}_{day + 1}')
                else:
                    x[train, cross, day] = model.addVar(vtype= BINARY, name=f'x_{train}_{cross}_{day + 1}')
# y: train assignment to repair type on day : y[train, type, day] = 1 if train assigned to type on day, 0 otherwise
# denote: Y_{i, k, t}
y = {}
for day in range(num_days):
    for train in train_set:
        for type in repair_types:
            if day == 0:
                value = day1_schedule.loc[type][train]
                y[train, type, day] = model.addVar(vtype=BINARY, lb = value, ub = value, name=f'y_{train}_{type}_{day + 1}')
            else:
                y[train, type, day] = model.addVar(vtype=BINARY, name=f'y_{train}_{type}_{day + 1}')
# z: remaining distance to repair type of train after day : z[train, type, day]
# d: remaining days to repair type Z of train after day : d[train, day]
z = {}
for train in train_set:
    for type in repair_types:
        for day in range(num_days):
            if day == 0:
                # set solution value of day0
                remain_distance = train_data.loc[train][type + '剩余里程']
                z[train, type, day] = model.addVar(vtype=CONTINUOUS, lb=remain_distance, ub=remain_distance, name=f'z_{train}_{type}_{day + 1}')
            else:  
                z[train, type, day] = model.addVar(vtype=CONTINUOUS, lb=0, ub=reset_distance[type], name=f'z_{train}_{type}_{day + 1}')
d = {}
for train in train_set:
    for day in range(num_days):
        if day == 0:
            # set solution value of day0
            remain_Z_days = train_data.loc[train]['Z剩余天数']
            d[train, day] = model.addVar(vtype=CONTINUOUS, lb=remain_Z_days, ub=remain_Z_days, name=f'd_{train}_{day}')
        else:
            d[train, day] = model.addVar(vtype=CONTINUOUS, lb=0, ub=reset_days['Z'], name=f'd_{train}_{day}')

Cardinal Optimizer v7.2.4. Build date Dec  6 2024
Copyright Cardinal Operations 2024. All Rights Reserved



In [15]:
# Add constraints
# 1. Repair capability constraints: \sum\limits_{i \in I} Y_{i, k, t} \leq C_{k, t} \for k, t.
# 1）每日各检修项目检修能力限制，安排检修计划不能突破检修能力
for day in range(num_days):
    model.addConstr(sum(y[train, 'Z', day] for train in train_set) <= repair_capability.loc['Z']['day' + str(day + 1)],
                    name=f'ZRepairCapacity_{day + 1}')
    model.addConstr(sum(y[train, 'L', day] for train in train_set) <= repair_capability.loc['L']['day' + str(day + 1)],
                    name=f'LRepairCapacity_{day + 1}')

# 2. Route assignment constraints : sum_{i} X[i, c, t] == 1  \for i,t
# 2) 运检计划不能有交路缺编，即没有车组执行
for route in routes_set:
    for cross in route_crosses[route]:
        for day in range(num_days):
            if distance_data.loc[cross]['day' + str(day + 1)] == 1:
                model.addConstr(sum(x[train, cross, day] 
                                    for train in train_set if route in candidate_routes[train]) 
                                    == 1,
                            name=f'CrossAssignment_{cross}_{day + 1}')

# 车组状态：同一时刻同一车组至多安排一个交路任务，或被检修
for train in train_set:
    for day in range(num_days):
        for type in ['Z', 'L']:
            model.addConstr(
                sum(x[train, cross, day]
                            for route in candidate_routes[train]
                            if route in route_crosses.keys()
                            for cross in route_crosses[route]
                            ) <= 1 - y[train, type, day],
                name=f'TrainState_{train}_{type}_{day + 1}')
            # 是否可以同时执行多种检修？？
            # 否
            # model.addConstr(
            #     sum(y[train, type, day] for type in ['Z', 'L']) <= 1,
            #     name=f'TrainRepairState_{train}_{day}')

# 3. Remaining mileage and day constraints
# 4) 每辆车的每种检修项目的剩余可用里程数和天数不能突破
for train in train_set:
    for day in range(1, num_days):
        # update the remaining mileage and day
        # distances
        # Z
        model.addConstr(z[train, 'Z', day] >= z[train, 'Z', day - 1] - sum(x[train, cross, day] * distance_data.loc[cross]['distance']
                                                                           for route in candidate_routes[train]
                                                                           if route in route_crosses.keys()
                                                                           for cross in route_crosses[route]
                                                                           ),
                        name=f'ZRemainingDistance_{train}_{day + 1}_lb')
        model.addConstr(z[train, 'Z', day] <= z[train, 'Z', day - 1] - sum(x[train, cross, day] * distance_data.loc[cross]['distance']
                                                                           for route in candidate_routes[train]
                                                                           if route in route_crosses.keys()
                                                                           for cross in route_crosses[route]
                                                                           ) + reset_distance['Z'] * y[train, 'Z', day],
                        name=f'ZRemainingDistance_{train}_{day + 1}_ub')
        # L
        model.addConstr(z[train, 'L', day] >= z[train, 'L', day - 1] - sum(x[train, cross, day] * distance_data.loc[cross]['distance']
                                                                           for route in candidate_routes[train]
                                                                           if route in route_crosses.keys()
                                                                           for cross in route_crosses[route]
                                                                           ),
                        name=f'LRemainingDistance_{train}_{day + 1}_lb')
        model.addConstr(z[train, 'L', day] <= z[train, 'L', day - 1] - sum(x[train, cross, day] * distance_data.loc[cross]['distance']
                                                                           for route in candidate_routes[train]
                                                                           if route in route_crosses.keys()
                                                                           for cross in route_crosses[route]
                                                                           ) + reset_distance['L'] * y[train, 'L', day],
                        name=f'LRemainingDistance_{train}_{day + 1}_ub')
        # days
        # Z
        model.addConstr(d[train, day] >= d[train, day - 1] - sum(x[train, cross, day] 
                                                                 for route in candidate_routes[train]
                                                                 if route in route_crosses.keys()
                                                                 for cross in route_crosses[route]
                                                                 ),
                        name=f'RemainingDays_{train}_{day + 1}_lb')
        model.addConstr(d[train, day] <= d[train, day - 1] - sum(x[train, cross, day] 
                                                                 for route in candidate_routes[train]
                                                                 if route in route_crosses.keys()
                                                                 for cross in route_crosses[route]
                                                                 ) + reset_days['Z'] * y[train, 'Z', day],
                        name=f'RemainingDays_{train}_{day + 1}_ub')

# 4. Continuity constraints
# R_ID相同的交路需要从上到下连续执行
for train in train_set:
    for route in candidate_routes[train]:
        if route  not in route_crosses.keys():
            continue
        for cross_index in range(1, len(route_crosses[route])):
                cross = route_crosses[route][cross_index]
                pre_cross = route_crosses[route][cross_index - 1]
                for day in range(1, num_days): 
                    # x[i,r,t] == x[i,r-1,t-1]
                    model.addConstr(x[train, cross, day] == x[train, pre_cross, day - 1],
                                    name=f'Continuity_{train}_{cross}_{day + 1}')

In [16]:
# def set_initial_solution(result_benchmark):
#     for cross in cross_routes:
#         for day in range(num_days):
#             train = result_benchmark.loc[cross, 'Day' + str(day+1)]
#             if train == '无任务':
#                 continue
#             x[train, cross, day].Start = 1

#     for type in repair_types:
#         for day in range(num_days):
#             trainstr = result_benchmark.loc[type, 'Day' + str(day+1)]
#             if pd.isna(trainstr):
#                 continue
#             trains = trainstr.split(',')
#             for train in trains:
#                 y[train, type, day].Start = 1

# set_initial_solution(result_benchmark)

In [17]:
def get_solution():
    # Output results
    # 创建一个空的 DataFrame，列名为 Day1 到 Day8，索引为交路
    columns = ['Day1', 'Day2', 'Day3', 'Day4', 'Day5', 'Day6', 'Day7', 'Day8']
    result_df = pd.DataFrame(index=cross_routes + repair_types + ['后备车组'], columns=columns)

    # 将 NaN 值替换为 '无任务'
    result_df.fillna('无任务', inplace=True)

    # 填充 DataFrame 结果
    # cross route schedule
    flags = {}
    for day in range(num_days):
        for train in train_set:
            flags[train, day] = False
            for route in candidate_routes[train]:
                if route not in route_crosses.keys():
                        continue
                for cross in route_crosses[route]:
                    if x[train, cross, day].X > 0.5:
                            flags[train, day] = True
                            result_df.loc[cross, f'Day{day+1}'] = f'{train}'
    # repair type schedule
    for day in range(num_days):
        for type in repair_types:
            for train in train_set:
                if result_df.loc[type, f'Day{day+1}'] == '无任务':
                    result_df.loc[type, f'Day{day+1}'] = ""
                if y[train, type, day].X > 0.5:
                    flags[train, day] = True
                    if result_df.loc[type, f'Day{day+1}'] != "":
                        result_df.loc[type, f'Day{day+1}'] += ","
                    result_df.loc[type, f'Day{day+1}'] += (f'{train}')
    # remaining trains
    for day in range(num_days):
        if result_df.loc['后备车组', f'Day{day+1}'] == '无任务':
            result_df.loc['后备车组', f'Day{day+1}'] = ""
        for train in train_set:
            if flags[train, day] == False:
                if result_df.loc['后备车组', f'Day{day+1}'] != "":
                    result_df.loc['后备车组', f'Day{day+1}'] += ","     
                result_df.loc['后备车组', f'Day{day+1}'] += (f'{train}')

    # print(result_df)

    # 返回结果
    return result_df

In [18]:
def solve_problem_I(model):
    # Objective function
    # 1) 避免过修，即检修间隔尽量用足剩余里程和天数
    # 引入辅助变量
    #  z'[i, k, t] = z[i, k, t] * y[i, k, t]  <==>  z'[i, k, t] >= z[i, k, t] - reset_distance[t] * (1 - y[i, k, t])
    #  d'[i, t] = sum{k} d[i, t] * y[i, 'Z', t] <==>  d'[i, t] >= sum{k} d[i, t] - reset_distance[t] * (1 - y[i, 'Z', t])
    # min \sum{i, k, t} z'[i, k, t] + \sum{i, t} d'[i, t]
    zz = {}
    dd = {}
    for day in range(num_days):
        for train in train_set:
            dd[train, day] = model.addVar(vtype=CONTINUOUS, lb=0, ub=reset_days['Z'], name=f'dd_{train}_Z_{day + 1}')
            for type in repair_types:
                zz[train, type, day] = model.addVar(vtype=CONTINUOUS, lb=0, ub=reset_distance[type], name=f'zz_{train}_{type}_{day + 1}')
                
    for day in range(num_days):
        for train in train_set:
            model.addConstr(dd[train, day] >= d[train, day] - reset_days['Z'] * (1 - y[train, 'Z', day]), name=f'dd_{train}_Z_{day + 1}')
            for type in repair_types:
                model.addConstr(zz[train, type, day] >= z[train, type, day] - reset_distance[type] * (1 - y[train, type, day]), name=f'zz_{train}_{type}_{day + 1}')
                
    # 设置目标函数
    objective1 = sum(zz[train, type, day] 
                    for train in train_set 
                    for type in repair_types 
                    for day in range(num_days)) + sum(dd[train, day] 
                                                    for train in train_set 
                                                    for day in range(num_days))
    model.setObjective(objective1, sense=MINIMIZE)

    # Optimize model
    # model.write("Objective1.lp")
    
    if solver == 'copt':
        model.solve()
    elif solver == 'gurobi':
        model.optimize()

    print("Objective1: %g" % model.objVal)

    result1 = get_solution()
    metrics_result1 = evaluate_schedule(result1)
    # print(metrics_result1)

    return result1

# result1 = solve_problem_I(model)

In [19]:
def solve_problem_II(model):
    # 2) 换车次数少，希望车组V执行连续交路R之后，继续再次执行连续交路R
    # 引入辅助变量：xx[train, route] = max{x[train, cross, day] for cross in cross_routes[route]}, 1 表示车组V在第day天执行了连续交路R
    # xx[train, route] = max{x[train, cross, day]  <==> xx[train, route, day] >= x[train, cross, day] for cross in cross_routes[route]
    xx = {}
    for train in train_set:
        for route in candidate_routes[train]:
            for day in range(num_days):
                xx[train, route] = model.addVar(vtype=BINARY, name=f'xx_{train}_{route}')
    
    # 添加辅助约束
    for train in train_set:
        for route in candidate_routes[train]:
            if route in route_crosses.keys():
                for cross in route_crosses[route]:
                    for day in range(num_days):
                        model.addConstr(xx[train, route] >= x[train, cross, day], name=f'xx_{train}_{route}_{day + 1}')

    objective2 = sum(xx[train, route] 
                    for train in train_set 
                    for route in candidate_routes[train])
                        
    model.setObjective(objective2, sense=MINIMIZE)
    
    # Optimize model
    # model.write("Objective2.lp")

    if solver == 'copt':
        model.solve()
    elif solver == 'gurobi':
        model.optimize()
    print("Objective2: %g" % model.objVal)

    result2 = get_solution()
    metrics_result2 = evaluate_schedule(result2)
    # print(metrics_result2)

    return result2

# result2 = solve_problem_II(model)

In [20]:
def solve_problem_III(model):
    # 3) 每日检修工作量均匀
    # 最小化检修工作量极差 min(max_t(sum_{i, k}y[i,k,t]) - min_t(sum_{i, k}y[i,k,t]))
    # 引入辅助变量: 
    #  min yy_max - yy_min
    #   yy_max = max_t(sum_{i, k}y[i,k,t])   <==>  yy_max >= sum_{i, k}y[i,k,t] for all t
    #   yy_min = min_t(sum_{i, k}y[i,k,t])   <==>  yy_min <= sum_{i, k}y[i,k,t] for all t
    yy_max = model.addVar(vtype=CONTINUOUS, lb = 0, ub = INFINITY, name = "yy_max")
    yy_min = model.addVar(vtype=CONTINUOUS, lb = 0, ub = INFINITY, name = "yy_min")
    
    for day in range(num_days):
        model.addConstr(yy_max >= sum(y[train, type, day] for train in train_set for type in repair_types), name = "yy_max_%d"%day)
        model.addConstr(yy_min <= sum(y[train, type, day] for train in train_set for type in repair_types), name = "yy_min_%d"%day)
    objective3 = yy_max - yy_min

    model.setObjective(objective3, sense=MINIMIZE)

    # Optimize model
    # model.write("Objective3.lp")

    if solver == 'copt':
        model.solve()
    elif solver == 'gurobi':
        model.optimize()

    print("Objective3: %g" % model.objVal)

    result3 = get_solution()
    metrics_result3 = evaluate_schedule(result3)
    # print(metrics_result3)
    
    return result3

# result3 = solve_problem_III(model)

In [ ]:
# 对比评价指标
metrics_benchmark = evaluate_schedule(result_benchmark)
print("Benchmark: ", metrics_benchmark)
# Objective 1
result1 = solve_problem_I(model)
metrics_result1 = evaluate_schedule(result1)
print("Objective I: ", metrics_result1)
# Objective 2
result2 = solve_problem_II(model)
metrics_result2 = evaluate_schedule(result2)
print("Objective II: ", metrics_result2)
# Objective 3
result3 = solve_problem_III(model)
metrics_result3 = evaluate_schedule(result3)
print("Objective III: ", metrics_result3)

metrics_df = pd.DataFrame([metrics_benchmark, metrics_result1, metrics_result2, metrics_result3], index=['Benchmark', 'Objective I', 'Objective II', 'Objective III'])

# 将结果写入到Excel文件的不同Sheet中
with pd.ExcelWriter('optimization_results.xlsx') as writer:
    result_benchmark.to_excel(writer, sheet_name='甘特图(benchmark)')
    result1.to_excel(writer, sheet_name='甘特图-I')
    result2.to_excel(writer, sheet_name='甘特图-II')
    result3.to_excel(writer, sheet_name='甘特图-III')
    metrics_df.to_excel(writer, sheet_name='评价指标')